In [213]:
from lxml import etree
import requests
import pandas as pd
import re
import uuid

In [249]:
path = "laptop_data.csv"
csv_head = ["uuid","name","price","processor","memory","hard_drive","graphics","img"]
pd.DataFrame(columns=csv_head).to_csv(path, index=False)

In [3]:
# crawler for hp official website
class HPCrawler:
  def __init__(self, url='https://store.hp.com/sg-en/default/laptops-tablets.html?product_list_limit=30'):
    self.url = url
    self.html = etree.HTML(requests.get(url).text)
    self.product_urls = self.html.xpath('//*[@id="category.product.list"]//ol[@class="products list items product-items"]/li//a[@class="product-item-link"]/@href')
  
  ## refrash the url
  def refrashURL(self, url):
    self.url = url
    self.html = etree.HTML(requests.get(url).text)
    self.product_urls = self.html.xpath('//*[@id="category.product.list"]//ol[@class="products list items product-items"]/li//a[@class="product-item-link"]/@href')

  def getProductsCount(self):
    return len(self.product_urls)

  def getProductInfo(self, index):
    item=[]
    url = self.product_urls[index]
    html = etree.HTML(requests.get(url).text)

    ## get name
    try:
      name = html.xpath('//*[@id="maincontent"]//span[@class="base"]/text()')[0]
    except IndexError:
      name ='NA'
      item.append(name)
    else:
      item.append(name)

    ## get price
    try:
      price = html.xpath('//div[@class="product-info-price"]//span[@class="price"]/text()')[0]
    except IndexError:
      price = 'NA'
      item.append(price)
    else:
      item.append(price)

    ## get processor info
    try:
      processor = str(html.xpath('//*[@id="product-attribute-specs-table"]/tbody//td[@data-th="Processor"]/text()')[0]).replace('®','').replace('™','')
    except IndexError:
      processor = 'NA'
      item.append(processor)
    else:
      item.append(processor)

    ## get memory info
    try:
      memory = str(html.xpath('//*[@id="product-attribute-specs-table"]/tbody//td[@data-th="Memory"]/text()')[0]).replace('®','').replace('™','')
    except IndexError:
      memory = 'NA'
      item.append(memory)
    else:
      item.append(memory)

    ## get hard drive info
    try:
      hard_drive = str(html.xpath('//*[@id="product-attribute-specs-table"]/tbody//td[@data-th="Hard drive description"]/text()')[0]).replace('®','').replace('™','')
    except IndexError:
      hard_drive = 'NA'
      item.append(hard_drive)
    else:
      item.append(hard_drive)

    ## get graphics info
    try:
      graphics = str(html.xpath('//*[@id="product-attribute-specs-table"]/tbody//td[@data-th="Graphics"]/text()')[0]).replace('®','').replace('™','')
    except IndexError:
      graphics='NA'
      item.append(graphics)
    else:
      item.append(graphics)

    return item

In [ ]:
HP = HPCrawler()

In [ ]:
HP.refrashURL('https://store.hp.com/sg-en/default/laptops-tablets.html?p=2&product_list_limit=30')

In [8]:
count = HP.getProductsCount()
results=[]
for i in range(count):
  results.append(HP.getProductInfo(i))

NameError: ignored

In [ ]:
df = pd.DataFrame(data=results)
df.to_csv(path, mode='a', index=False, header=False)

In [253]:
# crawler for dell official website
class DellCrawler:
  def __init__(self,url='https://www.dell.com/en-sg/shop/laptops-and-2in1-pcs/sr/laptops?sortBy=price-ascending'):
    self.url = url
    self.html = etree.HTML(requests.get(url).text)
    self.product_urls = self.html.xpath('//*[@id="ps-wrapper"]/article//h3[@class="ps-title"]/a/@href')
  
  def refrashURL(self, url):
    self.url = url
    self.html = etree.HTML(requests.get(url).text)
    self.product_urls = self.html.xpath('//*[@id="ps-wrapper"]/article//h3[@class="ps-title"]/a/@href')

  def getProductsCount(self):
    return len(self.product_urls)

  def getgetProductInfo(self, index):
    item=[]
    url = 'https:'+self.product_urls[index]
    html = etree.HTML(requests.get(url).text)

    ## get uuid
    id = str(uuid.uuid1())
    item.append(id)
    ## get name
    try:
      name = html.xpath('//*[@id="cf-body"]//h1[@class="cf-pg-title"]/span/text()')[0]
      print(name)
    except IndexError:
      name ='NA'
      item.append(name)
    else:
      item.append(name)

    ## get price
    price = html.xpath('//*[@id="cf-strike-through-price"]/div[2]/div/text()')
    if len(price)!=0:
      price = price[0].strip()
    else:
      try:
        price = html.xpath('//*[@id="cf-strike-through-price"]/div/div/text()')[0].strip()
      except IndexError:
        price='NA'
    item.append(price)

    ## get processor info
    try:
      processor = html.xpath('//*[@id="cf-body"]//ul[@class="cf-hero-bts-list"]/li[1]/p/text()')[0].strip().replace('®','').replace('™','')
      proc_intel = re.search(r'Intel', processor)
      if proc_intel != None:
        processor = re.search(r'Intel.*i\d*-[A-Z0-9]*', processor)[0]
      else:
        processor = 'NA'
    except IndexError:
      processor = 'NA'
      item.append(processor)
    else:
      item.append(processor)

    ## get memory info
    try:
      memory = html.xpath('//*[@id="cf-body"]//ul[@class="cf-hero-bts-list"]/li[4]/p/text()')[0].strip().replace('®','').replace('™','')
      memory = re.match(r'\d*', memory)[0]
    except IndexError:
      memory = 'NA'
      item.append(memory)
    else:
      item.append(memory)

    ## get hard drive info
    try:
      hard_drive = html.xpath('//*[@id="cf-body"]//ul[@class="cf-hero-bts-list"]/li[5]/p/text()')[0].strip().replace('®','').replace('™','')
      drive_ssd = re.search(r'Solid|SSD', hard_drive)
      drive_hdd = re.search(r'Hard', hard_drive)
      if drive_ssd!=None and drive_hdd!=None:
        hard_drive_SSD = (re.search(r'\d*GB', hard_drive)[0]).replace(' ','')+'_SSD'
        hard_drive_HDD = (re.search(r'\d*TB', hard_drive)[0]).replace(' ','')+'_HDD'
        hard_drive = hard_drive_SSD+'+'+hard_drive_HDD
      elif drive_ssd != None:
        hard_drive = (re.search(r'\d*.*[G|T]B', hard_drive)[0]).replace(' ','')+'_SSD'
      elif drive_hdd != None:
        hard_drive = (re.search(r'\d*.*[G|T]B', hard_drive)[0]).replace(' ','')+'_HDD'
      else:
        hard_drive = 'NA'
    except IndexError:
      hard_drive = 'NA'
      item.append(hard_drive)
    else:
      item.append(hard_drive)

    ## get graphics info
    try:
      graphics = html.xpath('//*[@id="cf-body"]//ul[@class="cf-hero-bts-list"]/li[3]/p/text()')[0].strip().replace('®','').replace('™','')
      gra_NV = re.search(r'NVIDIA', graphics)
      gra_In = re.search(r'Intel',graphics)
      if gra_NV != None:
        gra_NV = re.search(r'NVIDIA [a-zA-Z]* [A-Z0-9]*', graphics)[0]
        if re.search(r'\d',gra_NV)==None:
          gra_NV = re.search(r'NVIDIA [a-zA-Z]* [A-Z]* [0-9]* [a-zA-Z]*', graphics)[0]
        if gra_NV[-5:]==' with':
          gra_NV = gra_NV[:-5]
        if re.search(r'Max-Q', graphics)!=None:
          gra_NV = gra_NV+", Max-Q"
        graphics = gra_NV
      elif gra_In !=None:
        try:
          gra_In = re.match(r'Intel.*Graphics',graphics)[0]
        except TypeError:
          graphics='NA'
        else:
          graphics = gra_In
      else:
        graphics='NA'
    except IndexError:
      graphics='NA'
      item.append(graphics)
    else:
      item.append(graphics)

    ## get img
    img = html.xpath('//*[@id="cf_hero_image_wrap"]/img/@src')
    if len(img)!=0:
      img='https:'+img[0]
    else:
      try:
        img = html.xpath('//*[@id="cf_hero_image_wrap"]/div/div/img/@src')[0]
      except IndexError:
        img='NA'
      else:
        img = 'https:'+img
    item.append(img)

    return item

In [254]:
Dell = DellCrawler()
count = Dell.getProductsCount()
dell=[]
for i in range(count):
  dell.append(Dell.getgetProductInfo(i))
dell_df = pd.DataFrame(data=dell)
dell_df.to_csv(path, mode='a', index=False, header=False)

Inspiron 14 3000 (3493)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 14 5000 Laptop (5402)
Inspiron 15 3000 (3593)
New Inspiron 14 5000 Laptop (5402)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 15 5000 Laptop (5502)
New Inspiron 15 5000 Laptop (5501)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 15 5000 Laptop (5502)
New Inspiron 14 5000 Laptop (5402)
New Inspiron 15 5000 Laptop (5501)


In [255]:
url = 'https://www.dell.com/en-sg/shop/laptops-and-2in1-pcs/sr/laptops?page='
for i in range(2,10):
  temp_url = url+str(i)+'&sortBy=price-ascending'
  Dell.refrashURL(temp_url)
  count = Dell.getProductsCount()
  dell=[]
  for i in range(count):
    dell.append(Dell.getgetProductInfo(i))
  dell_df = pd.DataFrame(data=dell)
  dell_df.to_csv(path, mode='a', index=False, header=False)

New Inspiron 14 5000 Laptop (5401)
Inspiron 13 5000 Laptop (5391)
New Inspiron 13 5300 Laptop
New Inspiron 14 5000 2-in-1 Laptop (5406)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 14 5000 2-in-1 Laptop (5406)
New Inspiron 15 5000 Laptop (5502)
New Inspiron 15 7501 Laptop
New G3 15 3500 Laptop
New Inspiron 15 5000 Laptop (5501)
New Inspiron 14 5000 2-in-1 Laptop
New Inspiron 14 5000 Laptop (5402)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 15 5000 Laptop (5502)
New Inspiron 15 5000 Laptop (5501)
New Inspiron 14 5000 Laptop (5402)
New Inspiron 14 5000 Laptop (5401)
New Inspiron 14 5000 Laptop (5401)
New Inspiron 15 5000 Laptop (5501)
New Inspiron 13 5300 Laptop
New Inspiron 14 5000 2-in-1 Laptop (5406)
Inspiron 13 5000 Laptop (5391)
New Inspiron 13 5300 Laptop
Inspiron 13 5000 Laptop (5391)
New Inspiron 13 5000 Laptop (5301)
New Inspiron 14 5000 2-in-1 Laptop (5406)
New Inspiron 15 5000 Laptop (5502)
New Inspiron 15 7501 Laptop
New Inspiron 15 5000 Laptop (5501)
New Inspiron 14 5

In [183]:
count = Dell.getProductsCount()
dell=[]
for i in range(count):
  dell.append(Dell.getgetProductInfo(i))

In [184]:
dell_df = pd.DataFrame(data=dell)
dell_df.to_csv(path, mode='a', index=False, header=False)

In [ ]:
data = pd.read_csv('laptop_data.csv')
data.head()


,name,price,processor,memory,hard_drive,graphics
0,Inspiron 15 3000 (3593),"$1,198.99",Intel Core i7-1065G7,8.0,1TB_HDD,NVIDIA GeForce MX230 with 2GB GDDR5 graphics m...
1,New Inspiron 14 5000 Laptop (5401),"$1,348.99",Intel Core i5-1035G1,8.0,512GB_SSD,NVIDIA GeForce MX330 with 2GB GDDR5 graphics m...
2,New Inspiron 14 5000 Laptop (5402),"$1,248.99",Intel Core i5-1135G7,8.0,512GB_SSD,Intel Iris Xe Graphics with shared graphics me...
3,New Inspiron 13 5000 Laptop (5301),"$1,249.00",Intel Core i5-1135G7,8.0,256GB_SSD,Intel Iris Xe Graphics with shared graphics me...
4,New Inspiron 14 5000 Laptop (5401),"$1,348.99",Intel Core i5-1035G1,8.0,512GB_SSD,NVIDIA GeForce MX330 with 2GB GDDR5 graphics m...
